In [1]:
# Import Libraries
import tweepy as tw
import configparser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import re
import nltk
import datetime
import pytz
from dateutil.relativedelta import relativedelta

# Authentication

In [29]:
keys = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\TW API Keys\apikeys.csv")

my_api_key = keys["Keys"][0]
my_api_secret = keys["Keys"][1]

my_access_token = keys["Keys"][2]
my_access_token_secret = keys["Keys"][3]

In [30]:
# your Twitter API key ans access token
api_key = my_api_key
api_secret = my_api_secret

access_token = my_access_token
access_token_secret = my_access_token_secret

In [31]:
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
auth.set_access_token(access_token, access_token_secret)

In [32]:
api = tw.API(auth, wait_on_rate_limit=True)

## Variables

sc_name = "el33th4xor"

In [33]:
user_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input/Top 20 Cryptocurrencies and Related TW Accounts.csv", encoding= 'unicode_escape')
user_df

,Coin,ID,Owner/Influence,Name,Screen_name,Status,Why did I choose this person?
0,Ethereum,ethereum,Vitalik Buterin,vitalik.eth,VitalikButerin,co-founder,he is also known in crypto world
1,BNB,binancecoin,Changpeng Zhao,CZ ??Binance,cz_binance,CEO,NaN
2,XRP,ripple,Brad Garlinghouse,Brad Garlinghouse,bgarlinghouse,CEO,NaN
3,ADA(Cardano),cardano,Charles Hoskinson,Charles Hoskinson,IOHK_Charles,CEO/co-founder,NaN
4,DOGE,dogecoin,Elon Musk,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
5,MATIC(Polygon),matic-network,Sandeep Nailwal,Sandeep | Polygon ????3??,sandeepnailwal,co-founder,he seems more active than other co-founders
6,DOT(Polkadot),polkadot,WEB3 Foundation,Web3 Foundation,Web3foundation,foundation,there is no account of CEO or sth
7,HEX,hex,Richard Heart,Richard Heart,RichardHeartWin,CEO,NaN
8,WTRX,wrapped-tron,Justin Sun,H.E. Justin Sun,justinsuntron,founder,same with TRX!
9,SHIB,shiba-inu,Elon Musk,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."


In [34]:
endDate = datetime.datetime.now()
startDate = endDate - relativedelta(days = 3) #days, months, years can be used to calculate

utc=pytz.UTC

startDate = utc.localize(startDate) 
endDate = utc.localize(endDate)

In [35]:
limit = 200

# Getting Liked Tweets

In [36]:
def get_liked_tweets(sc_name):
    
    liked_tweets = []
    get_liked_tweets = api.get_favorites(screen_name = sc_name)
    for tweet in get_liked_tweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            liked_tweets.append(tweet)


    while (get_liked_tweets[-1].created_at > startDate):
        get_liked_tweets = api.get_favorites(screen_name = sc_name, max_id = get_liked_tweets[-1].id)
        for tweet in get_liked_tweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                liked_tweets.append(tweet)
    return liked_tweets

In [37]:
columns = ["Tweet_id", "Name", "Screen_name", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]

In [38]:
def categorize_liked_tweets(liked_tweets):

    data = []
    term_rt = "RT"
    for tweet in liked_tweets:

        #for consecutive tweets, next one is shown primaryly in data frame, I can check with using hours and minutes if necessary
        date = tweet.created_at
        date = date.strftime('%Y-%m-%d')

        #retweet and favorite counts
        number_of_retweet = tweet.retweet_count
        number_of_favorite = tweet.favorite_count


        #it's gonna be direct tweet if all the checks below fail
        category_specified = "direct"

        #retweet check
        if term_rt in tweet.text.split():

            category_specified = "retweet"

        #quote(rt with comment) check
        elif tweet.is_quote_status is True:
            category_specified = "quote"

        #reply check
        elif tweet.in_reply_to_status_id is not None:
            category_specified = "reply"

        data.append([tweet.id, tweet.user.name, tweet.user.screen_name, tweet.text, date, category_specified, number_of_retweet, number_of_favorite])
    
    df_met = pd.DataFrame(data, columns = columns)
    return df_met 

In [39]:
df = pd.DataFrame(columns = columns)
for x in user_df.Screen_name:
    
    taken_liked_tweets = get_liked_tweets(x)
    df_met_2 = categorize_liked_tweets(taken_liked_tweets)
    df = df.append(df_met_2)

In [40]:
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True, drop = True)

In [41]:
df.shape

(503, 8)

In [42]:
#null check --> if it is "False", we all ok
df.isna().any().any()

False

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Tweet_id     503 non-null    object
 1   Name         503 non-null    object
 2   Screen_name  503 non-null    object
 3   Tweet        503 non-null    object
 4   Date         503 non-null    object
 5   Category     503 non-null    object
 6   N_of_RT      503 non-null    object
 7   N_of_fav     503 non-null    object
dtypes: object(8)
memory usage: 31.6+ KB


In [44]:
#duplication check
df[df.duplicated(keep=False)].shape[0]

0

In [45]:
pd.set_option("max_rows", None)
df.to_excel(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\liked_tweets.xlsx")
df.head()

,Tweet_id,Name,Screen_name,Tweet,Date,Category,N_of_RT,N_of_fav
0,1603434498460307461,Bryan Caplan,bryan_caplan,"""You seem much more worried about Swedes losin...",2022-12-15,direct,0,22
1,1603430930315550728,S.J.F.,S_J_Foelsing,@XiXiDu @robinhanson No system of governance h...,2022-12-15,reply,0,2
2,1603405631385468930,S.J.F.,S_J_Foelsing,@robinhanson stable governance,2022-12-15,reply,0,6
3,1603364102595829762,Alex Godofsky,AlexGodofsky,This is a very common left-wing argument but s...,2022-12-15,direct,7,194
4,1603153716500013056,suzuha ⚡🌙,dystopiabreaker,@VitalikButerin note that the recovery mechani...,2022-12-14,reply,1,51
